<a href="https://colab.research.google.com/github/cuongtm2/machine-learning/blob/master/TF_training_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.7.0
!pip install tensorflow-datasets==4.4.0 
!pip install Pillow==8.4.0 
!pip install pandas==1.3.4 
!pip install numpy==1.21.4 
!pip install scipy==1.7.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 665.5 MB 235 kB/s
     |████████████████████████████████| 463 kB 7.6 MB/s 
     |████████████████████████████████| 1.3 MB 43.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
Looking in indexes: http

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.3 MB 10.6 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 7.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Build and train a neural network to predict sunspot activity using
# the Sunspots.csv dataset.
#
# Your neural network must have an MAE of 0.12 or less on the normalized dataset
# for top marks.
#
# Code for normalizing the data is provided and should not be changed.
#
# At the bottom of this file, we provide  some testing
# code in case you want to check your model.

# Note: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

series = np.array(sunspots)

In [ ]:
# This is the normalization function
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

# The data should be split into training and validation sets at time step 3000
# DO NOT CHANGE THIS CODE
split_time = 3000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
# mean = np.mean(series)
# std = np.std(series)
# series -= mean
# series /= std
# time = np.array(time_step)

# # The data should be split into training and validation sets at time step 3000
# # DO NOT CHANGE THIS CODE
# split_time = 3000
# time_train = time[:split_time]
# x_train = series[:split_time]
# time_valid = time[split_time:]
# x_valid = series[split_time:]

In [ ]:
# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
# Parameter 1
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000
train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                              shuffle_buffer=shuffle_buffer_size)
val_set = windowed_dataset(x_valid, window_size=window_size, batch_size=batch_size,
                              shuffle_buffer=shuffle_buffer_size)


# Parameter 2
# window_size = 100
# batch_size = 512
# shuffle_buffer_size = 1000
# train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
tf.keras.backend.clear_session()

# Model 2
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='causal', activation='relu', input_shape=[None, 1]),
  tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(16, activation="relu"),
  tf.keras.layers.Dense(1)
  # tf.keras.layers.Lambda(lambda x: x * 400)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 64)          384       
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, None, 64)          8256      
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 dense_1 (Dense)             (None, None, 32)          2

In [ ]:
patient = 10
save_path = '/content/drive/MyDrive/tf_certification/model5.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_mae",
                    mode="min",
                    verbose=1,
                    save_best_only=True)
    ]

In [ ]:
# Training model 2 - parameter 1
model.compile(loss=tf.keras.losses.Huber(), optimizer='adam', metrics=['mae'])
history = model.fit(train_set, epochs=200, verbose=1,validation_data=val_set,
                    callbacks=callbacks_list)

Epoch 1/200
     93/Unknown - 16s 81ms/step - loss: 0.0058 - mae: 0.0752
Epoch 1: val_mae improved from inf to 0.05803, saving model to /content/drive/MyDrive/tf_certification/model5.h5
93/93 [==============================] - 18s 108ms/step - loss: 0.0058 - mae: 0.0752 - val_loss: 0.0032 - val_mae: 0.0580 - lr: 0.0010
Epoch 2/200
93/93 [==============================] - ETA: 0s - loss: 0.0028 - mae: 0.0536
Epoch 2: val_mae improved from 0.05803 to 0.05701, saving model to /content/drive/MyDrive/tf_certification/model5.h5
93/93 [==============================] - 8s 90ms/step - loss: 0.0028 - mae: 0.0536 - val_loss: 0.0033 - val_mae: 0.0570 - lr: 0.0010
Epoch 3/200
93/93 [==============================] - ETA: 0s - loss: 0.0022 - mae: 0.0474
Epoch 3: val_mae improved from 0.05701 to 0.05276, saving model to /content/drive/MyDrive/tf_certification/model5.h5
93/93 [==============================] - 8s 85ms/step - loss: 0.0022 - mae: 0.0474 - val_loss: 0.0031 - val_mae: 0.0528 - lr: 0.0010

In [ ]:
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
model = tf.keras.models.load_model(save_path)
import math
def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast

split_time = 3000
window_size = 30
# YOUR CODE HERE
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

# To get the maximum score, your model must have an MAE OF .12 or less.
# When you Submit and Test your model, the grading infrastructure
# converts the MAE of your model to a score from 0 to 5 as follows:

test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
   score = 5

print(score)

5


In [ ]:
patient = 10
save_path = '/content/drive/MyDrive/tf_certification/model5_1.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_mae",
                    mode="min",
                    verbose=1,
                    save_best_only=True)
    ]

In [ ]:
# Training model 2 - parameter 1
model.compile(loss=tf.keras.losses.Huber(), optimizer='adam', metrics=['mae'])
history = model.fit(train_set, epochs=200, verbose=1,validation_data=val_set,
                    callbacks=callbacks_list)

Epoch 1/200
     93/Unknown - 16s 80ms/step - loss: 0.1879 - mae: 0.4942
Epoch 1: val_mae improved from inf to 0.29265, saving model to /content/drive/MyDrive/tf_certification/model5_1.h5
93/93 [==============================] - 18s 106ms/step - loss: 0.1879 - mae: 0.4942 - val_loss: 0.0775 - val_mae: 0.2927 - lr: 0.0010
Epoch 2/200
93/93 [==============================] - ETA: 0s - loss: 0.0753 - mae: 0.2920
Epoch 2: val_mae improved from 0.29265 to 0.24257, saving model to /content/drive/MyDrive/tf_certification/model5_1.h5
93/93 [==============================] - 8s 87ms/step - loss: 0.0753 - mae: 0.2920 - val_loss: 0.0618 - val_mae: 0.2426 - lr: 0.0010
Epoch 3/200
93/93 [==============================] - ETA: 0s - loss: 0.0554 - mae: 0.2461
Epoch 3: val_mae did not improve from 0.24257
93/93 [==============================] - 8s 83ms/step - loss: 0.0554 - mae: 0.2461 - val_loss: 0.0809 - val_mae: 0.2612 - lr: 0.0010
Epoch 4/200
93/93 [==============================] - ETA: 0s - los

In [ ]:
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
model = tf.keras.models.load_model(save_path)
import math
def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast

split_time = 3000
window_size = 30
# YOUR CODE HERE
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

# To get the maximum score, your model must have an MAE OF .12 or less.
# When you Submit and Test your model, the grading infrastructure
# converts the MAE of your model to a score from 0 to 5 as follows:

test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
   score = 5

print(score)

-15


## new

In [ ]:
# ==============================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative to its
# difficulty. So your Category 1 question will score significantly less than
# your Category 5 question.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure. You do not need them to solve the question.
#
# WARNING: If you are using the GRU layer, it is advised not to use the
# recurrent_dropout argument (you can alternatively set it to 0),
# since it has not been implemented in the cuDNN kernel and may
# result in much longer training times.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# TIME SERIES QUESTION
#
# Build and train a neural network to predict time indexed variables of
# the multivariate house hold electric power consumption time series dataset.
# Using a window of past 24 observations of the 7 variables, the model
# should be trained to predict the next 24 observations of the 7 variables.
#
# ==============================================================================
#
# ABOUT THE DATASET
#
# Original Source:
# https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption
#
# The original Individual House Hold Electric Power Consumption Dataset
# has Measurements of electric power consumption in one household with
# a one-minute sampling rate over a period of almost 4 years.
#
# Different electrical quantities and some sub-metering values are available.
#
# For the purpose of the examination we have provided a subset containing
# the data for the first 60 days in the dataset. We have also cleaned the
# dataset beforehand to remove missing values. The dataset is provided as a
# CSV file in the project.
#
# The dataset has a total of 7 features ordered by time.
# ==============================================================================
#
# INSTRUCTIONS
#
# Complete the code in following functions:
# 1. solution_model()
#
# Your code will fail to be graded if the following criteria are not met:
#
# 1. Model input shape must be (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7),
#    since the testing infrastructure expects a window of past N_PAST = 24
#    observations of the 7 features to predict the next N_FUTURE = 24
#    observations of the same features.
#
# 2. Model output shape must be (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7)
#
# 3. The last layer of your model must be a Dense layer with 7 neurons since
#    the model is expected to predict observations of 7 features.
#
# 4. Don't change the values of the following constants:
#    SPLIT_TIME, N_FEATURES, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT, in
#    solution_model() (See code for additional note on BATCH_SIZE).
#
# 5. Code for normalizing the data is provided - don't change it.
#    Changing the normalizing code will affect your score.
#
# 6. Code for converting the dataset into windows is provided - don't change it.
#    Changing the windowing code will affect your score.
#
# 7. Code for setting the seed is provided - don't change it.
#
# HINT: If you follow all the rules mentioned above and throughout this
# question while training your neural network, there is a possibility that a
# validation MAE of approximately 0.055 or less on the normalized validation
# dataset may fetch you top marks.


import urllib
import zipfile

import pandas as pd
import tensorflow as tf


# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# This function is used to map the time series dataset into windows of
# features and respective targets, to prepare it for training and
# validation. First element of the first window will be the first element of
# the dataset. Consecutive windows are constructed by shifting
# the starting position of the first window forward, one at a time (indicated
# by shift=1). For a window of n_past number of observations of all the time
# indexed variables in the dataset, the target for the window
# is the next n_future number of observations of these variables, after the
# end of the window.

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)



# This function loads the data from CSV file, normalizes the data and
# splits the dataset into train and validation data. It also uses
# windowed_dataset() to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network. This
# function returns the final trained model.

# COMPLETE THE CODE IN THIS FUNCTION
def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.
    download_and_extract_data()
    # Reads the dataset from the CSV.
    df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features at future time steps.
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 24  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
    N_FUTURE = 24  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

    # Code to define your model.
    model = tf.keras.models.Sequential([

        # ADD YOUR LAYERS HERE.

        # If you don't follow the instructions in the following comments,
        # tests will fail to grade your code:
        # The input layer of your model must have an input shape of:
        # (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7)
        # The model must have an output shape of:
        # (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7).
        # Make sure that there are N_FEATURES = 7 neurons in the final dense
        # layer since the model predicts 7 features.

        # HINT: Bidirectional LSTMs may help boost your score. This is only a
        # suggestion.

        # WARNING: If you are using the GRU layer, it is advised not to use the
        # recurrent_dropout argument (you can alternatively set it to 0),
        # since it has not been implemented in the cuDNN kernel and may
        # result in much longer training times.
        # tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='causal', activation='relu', input_shape=[None, 1]),
        tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(N_FEATURES)
    ])

    # Code to train and compile the model
    # optimizer =  # YOUR CODE HERE
    model.compile(
        # YOUR CODE HERE
        loss=tf.keras.losses.Huber(), optimizer='adam', metrics=['mae']

    )
    patient = 10
    save_path = '/content/drive/MyDrive/tf_certification/model5_new.h5'
    callbacks_list = [
        EarlyStopping(
            monitor='val_loss', 
            patience=patient, 
            mode='min', 
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor = 'val_loss', 
            factor = 0.5, 
            patience = patient / 2, 
            min_lr=1e-6,
            verbose=1,
            mode='min'
        ),
        ModelCheckpoint(save_path, 
                        monitor="val_mae",
                        mode="min",
                        verbose=1,
                        save_best_only=True)
        ]
    model.fit(train_set, epochs=200, verbose=1,validation_data=valid_set,callbacks=callbacks_list)

    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

# if _name_ == '_main_':
model = solution_model()
#     model.save("c5q4.h5")


# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
#def mae(y_true, y_pred):
#    return np.mean(abs(y_true.ravel() - y_pred.ravel()))
#
#
#def model_forecast(model, series, window_size, batch_size):
#    ds = tf.data.Dataset.from_tensor_slices(series)
#    ds = ds.window(window_size, shift=1, drop_remainder=True)
#    ds = ds.flat_map(lambda w: w.batch(window_size))
#    ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
#    forecast = model.predict(ds)
#    return forecast
#

# PASS THE NORMALIZED data IN THE FOLLOWING CODE

#rnn_forecast = model_forecast(model, data, N_PAST, BATCH_SIZE)
#rnn_forecast = rnn_forecast[SPLIT_TIME - N_PAST:-1, 0, :]

#x_valid = x_valid[:rnn_forecast.shape[0]]
#result = mae(x_valid, rnn_forecast)

Epoch 1/200
   1349/Unknown - 106s 69ms/step - loss: 0.0109 - mae: 0.0850
Epoch 00001: val_mae improved from inf to 0.06887, saving model to /content/drive/MyDrive/tf_certification/model5_new.h5
1349/1349 [==============================] - 134s 90ms/step - loss: 0.0109 - mae: 0.0850 - val_loss: 0.0087 - val_mae: 0.0689 - lr: 0.0010
Epoch 2/200
1349/1349 [==============================] - ETA: 0s - loss: 0.0083 - mae: 0.0712
Epoch 00002: val_mae improved from 0.06887 to 0.06253, saving model to /content/drive/MyDrive/tf_certification/model5_new.h5
1349/1349 [==============================] - 126s 93ms/step - loss: 0.0083 - mae: 0.0712 - val_loss: 0.0074 - val_mae: 0.0625 - lr: 0.0010
Epoch 3/200
1349/1349 [==============================] - ETA: 0s - loss: 0.0084 - mae: 0.0700
Epoch 00003: val_mae did not improve from 0.06253
1349/1349 [==============================] - 123s 91ms/step - loss: 0.0084 - mae: 0.0700 - val_loss: 0.0077 - val_mae: 0.0637 - lr: 0.0010
Epoch 4/200
1349/1349 [==